<a href="https://colab.research.google.com/github/EthanXC/RedditProject/blob/main/Copy_of_Version_0_0_RedditProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import numpy as np
import pandas as pd

In [16]:
!pip install datasets
# FYI the accellerate -u is very important to avoid version issues with torch
!pip install transformers
!pip install accelerate -U

In [17]:
from datasets import load_dataset

In [18]:
dataset = load_dataset("fddemarco/pushshift-reddit-comments", streaming = True)

Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

In [19]:
print(dataset)

IterableDatasetDict({
    train: IterableDataset({
        features: ['author', 'body', 'controversiality', 'created_utc', 'link_id', 'score', 'subreddit', 'subreddit_id', 'id'],
        n_shards: 50
    })
})


In [20]:
it = iter(dataset['train'])


In [21]:
extract_data = next(it)
#print(extract_data)

print(extract_data['body'])

isn ' t this against the first amendment ? doesn ' t the first amendment give us the right to assemble and protest ?


In [22]:
data = {
    'column1_name': ['body'],
    'column2_name': ['score'],
    'column3_name': ['subreddit']
}

df = pd.DataFrame(data)

In [23]:
# Initialize an empty DataFrame with columns
df = pd.DataFrame(columns=['body', 'score', 'subreddit'])

# Your code for iterating through the data and adding rows to the DataFrame
for i in range(10000):  # only 10,000 rows
    extract_data = next(it)
    body = extract_data['body']
    score = extract_data['score']
    subreddit = extract_data['subreddit']

    new_row_data = {'body': body, "score": score, "subreddit": subreddit}
    df.loc[len(df)] = new_row_data

    if i % 1000 == 0:
        print(i)
        print(new_row_data)


0
{'body': 'oil .', 'score': -4, 'subreddit': 'AskReddit'}
1000
{'body': 'is that a credible list ? how did they obtain their data ?', 'score': 1, 'subreddit': 'starcraft'}
2000
{'body': 'ok then : would you rather live in a street full of mps , or a street full of al - qaeda members , since this video seems to be implying that the threat level is the same .', 'score': -2, 'subreddit': 'unitedkingdom'}
3000
{'body': 'r / bad _ cop _ no _ latte coming soon .', 'score': 24, 'subreddit': 'Bad_Cop_No_Donut'}
4000
{'body': 'amen', 'score': 3, 'subreddit': 'treecomics'}
5000
{'body': "i remember that . . . it was a form message , wasn ' t it ? i used to end every one of my gm conversations , when they asked me ' is there anything else i can do to help you ' , with something along the lines of ' i know this is a shot in the dark , but could you possibly tell me an email address with which i might submit feedback and them promptly close the ticket , disallowing further communication with you ?

In [24]:
df.head()


,body,score,subreddit
0,oil .,-4,AskReddit
1,yeah nice . . . except for the fact that every...,5,politics
2,"bro , i figuratively just watched that episode .",2,gaming
3,this is why u can ' t vote,3,politics
4,"it ' s a wizard photograph , harry .",1,WTF


In [25]:
from transformers import AutoTokenizer, RobertaModel
import torch
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# Load DistilBERT model and tokenizer filling out the code below based on the documentation HINT: Click on on use in transformers and the code will be there
#model_name = "Roberta model"# code unnessisary but I like to use it to store the string name to make the code cleaner
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base")
model = RobertaModel.from_pretrained("roberta-base")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
nan_rows = df[df.isna().T.any().T]
nan_rows.head()

,body,score,subreddit


In [27]:
null_rows = df[df.isnull().any(axis=1)]
null_rows.head()

,body,score,subreddit


In [28]:
if np.isinf(df['score']).any():
  print("HELLO")

In [29]:
print(df.dtypes)

body         object
score         int64
subreddit    object
dtype: object


In [30]:
df.head()

,body,score,subreddit
0,oil .,-4,AskReddit
1,yeah nice . . . except for the fact that every...,5,politics
2,"bro , i figuratively just watched that episode .",2,gaming
3,this is why u can ' t vote,3,politics
4,"it ' s a wizard photograph , harry .",1,WTF


In [31]:
from sklearn.model_selection import train_test_split

#TODO: Complete this line using the train_test_split function and set the test size to .21
train_texts, val_texts, train_labels, val_labels = train_test_split(df['body'], df['subreddit'], test_size=0.21)


tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base")
# Reset the index
train_texts, train_labels = train_texts.reset_index(drop=True), train_labels.reset_index(drop=True)
val_texts, val_labels = val_texts.reset_index(drop=True), val_labels.reset_index(drop=True)

In [32]:
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, return_tensors = "pt")
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, return_tensors = "pt")

In [33]:
id2label = dict(enumerate(set(df['subreddit'])))

In [34]:
id2label

{0: 'SpideyMeme',
 1: 'askseddit',
 2: 'PS3',
 3: 'Iceland',
 4: 'ponypapers',
 5: 'cincinnati',
 6: 'BitcoinMining',
 7: 'Idliketobeatree',
 8: 'happy',
 9: 'SF4',
 10: 'comics',
 11: 'nosleep',
 12: 'Astrobiology',
 13: 'steamswap',
 14: 'GradSchool',
 15: 'QuotesPorn',
 16: 'applehelp',
 17: 'argentina',
 18: 'web_design',
 19: 'cumsluts',
 20: 'magicka',
 21: 'Bellingham',
 22: 'detroitlions',
 23: 'rva',
 24: 'askdrugs',
 25: 'spacedicks',
 26: 'canadients',
 27: 'radiohead',
 28: 'masculism',
 29: 'BabyBumps',
 30: 'geek',
 31: 'Random_Acts_Of_Pizza',
 32: 'movies',
 33: 'WeAreTheMusicMakers',
 34: 'treecomics',
 35: 'thewalkingdead',
 36: 'circlejerk',
 37: 'HeroesofNewerth',
 38: 'AmISexy',
 39: 'UFOs',
 40: 'Eve',
 41: 'washingtondc',
 42: 'religion',
 43: 'introvert',
 44: 'cordcutters',
 45: 'Fallout',
 46: 'torrents',
 47: 'gif',
 48: 'cscareerquestions',
 49: 'actuary',
 50: 'leafs',
 51: 'stopdrinking',
 52: 'saudiarabia',
 53: 'Libertarian',
 54: 'UTAustin',
 55: 'AlienB

In [35]:
id2label #number:word

{0: 'SpideyMeme',
 1: 'askseddit',
 2: 'PS3',
 3: 'Iceland',
 4: 'ponypapers',
 5: 'cincinnati',
 6: 'BitcoinMining',
 7: 'Idliketobeatree',
 8: 'happy',
 9: 'SF4',
 10: 'comics',
 11: 'nosleep',
 12: 'Astrobiology',
 13: 'steamswap',
 14: 'GradSchool',
 15: 'QuotesPorn',
 16: 'applehelp',
 17: 'argentina',
 18: 'web_design',
 19: 'cumsluts',
 20: 'magicka',
 21: 'Bellingham',
 22: 'detroitlions',
 23: 'rva',
 24: 'askdrugs',
 25: 'spacedicks',
 26: 'canadients',
 27: 'radiohead',
 28: 'masculism',
 29: 'BabyBumps',
 30: 'geek',
 31: 'Random_Acts_Of_Pizza',
 32: 'movies',
 33: 'WeAreTheMusicMakers',
 34: 'treecomics',
 35: 'thewalkingdead',
 36: 'circlejerk',
 37: 'HeroesofNewerth',
 38: 'AmISexy',
 39: 'UFOs',
 40: 'Eve',
 41: 'washingtondc',
 42: 'religion',
 43: 'introvert',
 44: 'cordcutters',
 45: 'Fallout',
 46: 'torrents',
 47: 'gif',
 48: 'cscareerquestions',
 49: 'actuary',
 50: 'leafs',
 51: 'stopdrinking',
 52: 'saudiarabia',
 53: 'Libertarian',
 54: 'UTAustin',
 55: 'AlienB

In [36]:
id2label_reversed = {v: k for k, v in id2label.items()}

In [37]:
id2label_reversed #word:number

{'SpideyMeme': 0,
 'askseddit': 1,
 'PS3': 2,
 'Iceland': 3,
 'ponypapers': 4,
 'cincinnati': 5,
 'BitcoinMining': 6,
 'Idliketobeatree': 7,
 'happy': 8,
 'SF4': 9,
 'comics': 10,
 'nosleep': 11,
 'Astrobiology': 12,
 'steamswap': 13,
 'GradSchool': 14,
 'QuotesPorn': 15,
 'applehelp': 16,
 'argentina': 17,
 'web_design': 18,
 'cumsluts': 19,
 'magicka': 20,
 'Bellingham': 21,
 'detroitlions': 22,
 'rva': 23,
 'askdrugs': 24,
 'spacedicks': 25,
 'canadients': 26,
 'radiohead': 27,
 'masculism': 28,
 'BabyBumps': 29,
 'geek': 30,
 'Random_Acts_Of_Pizza': 31,
 'movies': 32,
 'WeAreTheMusicMakers': 33,
 'treecomics': 34,
 'thewalkingdead': 35,
 'circlejerk': 36,
 'HeroesofNewerth': 37,
 'AmISexy': 38,
 'UFOs': 39,
 'Eve': 40,
 'washingtondc': 41,
 'religion': 42,
 'introvert': 43,
 'cordcutters': 44,
 'Fallout': 45,
 'torrents': 46,
 'gif': 47,
 'cscareerquestions': 48,
 'actuary': 49,
 'leafs': 50,
 'stopdrinking': 51,
 'saudiarabia': 52,
 'Libertarian': 53,
 'UTAustin': 54,
 'AlienBlue'

In [48]:
import torch as pt



class SpamDataset(pt.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels.values

    def __getitem__(self, idx):
        # TODO: finish this dictionary comprehention to get the correct tensors passed into our transformer.
        item = {key: pt.tensor(val[idx]) for key, val in self.encodings.items()}
        if len(self.labels) > 0:  # Check if labels array has any elements
            labels = self.labels[idx]
            print(labels)
            labels_converted = id2label_reversed[labels]
            print(labels_converted)
            item["labels"] = pt.tensor(labels_converted)
        return item
    def __len__(self):
        print(len(self.labels))
        return len(self.labels)

def list_of_dicts_to_dict_of_lists(d):
  dic = d[0]
  keys = dic.keys()
  values = [dic.values() for dic in d]
  return {k: list(v) for k,v in zip(keys, zip(*values))}
# TODO: Now create the train and val datasets with the class we created above. HINT look at the parameters SpamDataset has and think about what it does



In [49]:
Train_Dataset = SpamDataset(train_encodings, train_labels)


In [50]:
Val_Dataset = SpamDataset(val_encodings, val_labels)

In [51]:
!pip install datasets transformers torch evaluate nltk rouge_score

In [52]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate


In [53]:
from sklearn.metrics import accuracy_score

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
    # TODO: Look up on Hufggingface how to create a compute metrics function so we can return accuracy (using labels and predictions) and store the training information in a variable eval_pred


In [54]:
CUDA_LAUNCH_BLOCKING=1

In [55]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs, let's try 1 for now
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)



trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = Train_Dataset,
    eval_dataset = Val_Dataset,
    compute_metrics = compute_metrics
    #TODO: figure out what 5 params we need here. There should be 5 and the last is how we compute the metrics later . . .
)

7900


In [47]:
# TODO: Train the model (HINT: this is 1 short line of code just calling the trainer)
trainer.train()

7900
7900
7900
7900
7900
7900
7900
7900
7900
7900
7900
7900
7900
AskReddit
511
DebateReligion
639
beer
574
worldnews
647
translator
155
MensRights
738
gaming
453
facebook
118
AdviceAnimals
601
pics
535
AskReddit
511
guns
720
skyrim
325
CFB
65
TwoXChromosomes
429
fffffffuuuuuuuuuuuu
465


<ipython-input-38-90bfcc7ed591>:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: pt.tensor(val[idx]) for key, val in self.encodings.items()}


ValueError: The model did not return a loss from the inputs, only the following keys: last_hidden_state,pooler_output. For reference, the inputs it received are input_ids,attention_mask.

In [ ]:
#TODO use the evaluate method on the trainer to get and print the results. Feel free to look at Huggingface Docs
results = trainer.evaluate()

In [ ]:
# TODO: Save the model
trainer.save_model('my_model')

In [ ]:
# TODO: Now to finish off I want you to load the model you trained and saved and write a fake spam email for it mthen have the model classify it
model = AutoModelForSequenceClassification.from_pretrained("my_model")
msg = "i want help to fix my car"
msg_encodings = tokenizer(msg, truncation=True, padding=True, return_tensors = "pt")

outputs = model(**msg_encodings)
print(f"outputs: {outputs}")
